In [1]:
import preprocessor_end
import os
from shutil import copy
import pickle
import pandas as pd

In [2]:
#has three keys (0_label, 1_label, no_label)
#values: {movie_name:words, ....}
pickle_in = open('C:/Users/Abdo/Project/pickle_files/data_dictionary_before_removing_frequent_words.pkl','rb')
data_dictionary = pickle.load(pickle_in)


In [3]:
number_of_0_label_docs = len(data_dictionary["0_label"])
number_of_1_label_docs = len(data_dictionary["1_label"])

In [8]:
pandas_dict = {}

pandas_dict["rating"] = []
pandas_dict ["movie_id"] = []
pandas_dict ["movie_words"] = []



In [9]:
for label in list(data_dictionary.keys())[:2]:
    for movie_name in data_dictionary[label]:
        pandas_dict["rating"].append(label)
        pandas_dict["movie_words"].append(data_dictionary[label][movie_name])
        pandas_dict["movie_id"].append(movie_name)

In [10]:
# row: index
# columns : rating, movie_id, [word, word, ...]
data_frames = pd.DataFrame(data=pandas_dict, columns=["rating","movie_id","movie_words"])

In [11]:
#with open('C:/Users/Abdo/Project/pickle_files/data_dictionary_before_removing_frequent_words.pkl', 'wb') as output:
    #pickle.dump(data_dictionary, output)

In [12]:
#with open('C:/Users/Abdo/Project/pickle_files/pandas_data_frame.pkl', 'wb') as output:
    #pickle.dump(data_frames, output)

In [4]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/pandas_data_frame.pkl','rb')
data_frames = pickle.load(pickle_in)

In [6]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/word2vec_model_trained_150.pkl','rb')
model = pickle.load(pickle_in)

C:\Users\Abdo\PycharmProjects\final_project_1\venv\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
# list_of_lengths_and_id: [ (words_count, movie_id, rating), ..... ]
max = (0, "movie_name")
min = (100000000,"movie_name")
list_of_lengths_and_id = []

for index, row in data_frames.iterrows():
    movie_id = row["movie_id"]
    rating = row["rating"]
    length = len(row["movie_words"])
    list_of_lengths_and_id.append( (length, movie_id, rating) )

In [8]:
list_of_lengths_and_id.sort()

In [9]:
# find the max, and min number of words
min_number_of_words = list_of_lengths_and_id[0]
max_number_of_words = list_of_lengths_and_id[-1]

In [10]:
# find the average number of words
total = 0
for file in list_of_lengths_and_id:
    total = total + file[0]
average_number_of_words = total / len(list_of_lengths_and_id)

In [13]:
#from the word2vec model
vocabulary = set(model.wv.vocab)

In [17]:
dict_of_1_label_words = {}
dict_of_0_label_words = {}

In [18]:
# dict_of_0_label_words:
# key : word
# value: [number_of_total_occurrence, {movie_name:number_of_occurrence_in_movie, movie_name:number_of_occurrence_in_movie, .....}]
for movie_name in data_dictionary["0_label"]:
    for word in data_dictionary["0_label"][movie_name]:
        if word in dict_of_0_label_words:
            dict_of_0_label_words[word][0] = dict_of_0_label_words[word][0] + 1
            if(movie_name in dict_of_0_label_words[word][1]):
                (dict_of_0_label_words[word][1])[movie_name] = (dict_of_0_label_words[word][1])[movie_name] + 1
            else:
                (dict_of_0_label_words[word][1])[movie_name] = 1
        else:
            dict_of_0_label_words[word] = [1, {movie_name:1}]

In [19]:
for movie_name in data_dictionary["1_label"]:
    for word in data_dictionary["1_label"][movie_name]:
        if word in dict_of_1_label_words:
            dict_of_1_label_words[word][0] = dict_of_1_label_words[word][0] + 1
            if(movie_name in dict_of_1_label_words[word][1]):
                (dict_of_1_label_words[word][1])[movie_name] = (dict_of_1_label_words[word][1])[movie_name] + 1
            else:
                (dict_of_1_label_words[word][1])[movie_name] = 1
        else:
            dict_of_1_label_words[word] = [1, {movie_name:1}]

In [20]:
#with open('C:/Users/Abdo/Project/pickle_files/dict_of_1_label_words.pkl', 'wb') as output:
    #pickle.dump(dict_of_1_label_words, output)

In [21]:
#with open('C:/Users/Abdo/Project/pickle_files/dict_of_0_label_words.pkl', 'wb') as output:
    #pickle.dump(dict_of_0_label_words, output)

In [11]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/dict_of_1_label_words.pkl','rb')
dict_of_1_label_words = pickle.load(pickle_in)

In [12]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/dict_of_0_label_words.pkl','rb')
dict_of_0_label_words = pickle.load(pickle_in)

In [13]:
inappropriate_words = []
# key : inappropriate_word
#value : [total_occurences, {movie_id: occurenes_in_movie,movie_id: occurenes_in_movie,.....}]
inappropriate_words_in_0_label_movies = {}

In [14]:
file = open("C:/Users/Abdo/Project/Notes/inappropriate_words.txt","r")
for word in file.readlines():
    inappropriate_words.append(word.replace("\n",""))

In [15]:
#fill the values 
for inappropriate_word in inappropriate_words:
    if inappropriate_word in dict_of_0_label_words:
        inappropriate_words_in_0_label_movies[inappropriate_word] = dict_of_0_label_words[inappropriate_word]
    else:
        inappropriate_words_in_0_label_movies[inappropriate_word] = [0,[]]

In [17]:
#with open('C:/Users/Abdo/Project/pickle_files/inappropriate_words_in_0_label_movies.pkl', 'wb') as output:
    #pickle.dump(inappropriate_words_in_0_label_movies, output)

In [ ]:
#pickle_in = open('C:/Users/Abdo/Project/pickle_files/inappropriate_words_in_0_label_movies.pkl','rb')
#inappropriate_words_in_0_label_movies = pickle.load(pickle_in)

In [20]:
#total_words_in_all_movies
all_text_words = set(dict_of_1_label_words.keys())
all_text_words.update(set(dict_of_0_label_words.keys()))

In [21]:
common_words = set(dict_of_1_label_words.keys()).intersection(set(dict_of_0_label_words.keys()))

In [22]:
percentage_of_common_words = len(common_words)/len(all_text_words)

In [23]:
unique_words_in_0_label = set(dict_of_0_label_words.keys()).difference(common_words)
unique_words_in_1_label = set(dict_of_1_label_words.keys()).difference(common_words)

In [64]:
# key : word
# value : [ [total frequency, {movie_name: frequency_of_word_in_movie},..] , .... ] 
frequency_of_unique_words_in_0_label = {}
frequency_of_unique_words_in_1_label = {}

In [65]:
for movie_name in data_dictionary["0_label"]:
    for word in data_dictionary["0_label"][movie_name]:
        if word in unique_words_in_0_label:
            if word in frequency_of_unique_words_in_0_label:
                frequency_of_unique_words_in_0_label[word][0] = frequency_of_unique_words_in_0_label[word][0]+1
                if movie_name in frequency_of_unique_words_in_0_label[word][1]:
                    frequency_of_unique_words_in_0_label[word][1][movie_name] = \
                    frequency_of_unique_words_in_0_label[word][1][movie_name]+1
                else:
                    frequency_of_unique_words_in_0_label[word][1][movie_name] = 1
            else:
                frequency_of_unique_words_in_0_label[word] = [1, {movie_name:1}]
        

In [72]:
for movie_name in data_dictionary["1_label"]:
    for word in data_dictionary["1_label"][movie_name]:
        if word in unique_words_in_1_label:
            if word in frequency_of_unique_words_in_1_label:
                frequency_of_unique_words_in_1_label[word][0] = frequency_of_unique_words_in_1_label[word][0]+1
                if movie_name in frequency_of_unique_words_in_1_label[word][1]:
                    frequency_of_unique_words_in_1_label[word][1][movie_name] = \
                    frequency_of_unique_words_in_1_label[word][1][movie_name]+1
                else:
                    frequency_of_unique_words_in_1_label[word][1][movie_name] = 1
            else:
                frequency_of_unique_words_in_1_label[word] = [1, {movie_name:1}]

In [73]:
#with open('C:/Users/Abdo/Project/pickle_files/frequency_of_unique_words_in_1_label.pkl', 'wb') as output:
    #pickle.dump(frequency_of_unique_words_in_1_label, output)

In [74]:
#with open('C:/Users/Abdo/Project/pickle_files/frequency_of_unique_words_in_0_label.pkl', 'wb') as output:
    #pickle.dump(frequency_of_unique_words_in_0_label, output)

In [ ]:
#pickle_in = open('C:/Users/Abdo/Project/pickle_files/frequency_of_unique_words_in_0_label.pkl','rb')
#frequency_of_unique_words_in_0_label = pickle.load(pickle_in)

In [ ]:
#pickle_in = open('C:/Users/Abdo/Project/pickle_files/frequency_of_unique_words_in_1_label.pkl','rb')
#frequency_of_unique_words_in_1_label = pickle.load(pickle_in)

In [75]:
# [(frequency, word), ....]
list_of_uinque_0_label_words_and_frequencies = []
list_of_uinque_1_label_words_and_frequencies = []

In [76]:
for word in frequency_of_unique_words_in_0_label:
    list_of_uinque_0_label_words_and_frequencies.append( (frequency_of_unique_words_in_0_label[word][0], word) )

In [77]:
for word in frequency_of_unique_words_in_1_label:
    list_of_uinque_1_label_words_and_frequencies.append( (frequency_of_unique_words_in_1_label[word][0], word) )

In [78]:
list_of_uinque_0_label_words_and_frequencies.sort()
list_of_uinque_1_label_words_and_frequencies.sort()

In [81]:
#with open('C:/Users/Abdo/Project/pickle_files/list_of_uinque_0_label_words_and_frequencies.pkl', 'wb') as output:
    #pickle.dump(list_of_uinque_0_label_words_and_frequencies, output)

In [82]:
#with open('C:/Users/Abdo/Project/pickle_files/list_of_uinque_1_label_words_and_frequencies.pkl', 'wb') as output:
    #pickle.dump(list_of_uinque_1_label_words_and_frequencies, output)

In [24]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/list_of_uinque_1_label_words_and_frequencies.pkl','rb')
list_of_uinque_1_label_words_and_frequencies = pickle.load(pickle_in)

In [25]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/list_of_uinque_0_label_words_and_frequencies.pkl','rb')
list_of_uinque_0_label_words_and_frequencies = pickle.load(pickle_in)

In [87]:
# the percentage of bad words in every 0_labeled movie
# key : movie_id
# value : percentage_of_bad_words
percentage_of_bad_words_in_0_labeld_movies = {}

In [89]:
for movie_name in data_dictionary["0_label"]:
    movie_words_count = len(data_dictionary["0_label"][movie_name])
    number_of_bad_words = 0
    for bad_word in inappropriate_words_in_0_label_movies:
        if movie_name in inappropriate_words_in_0_label_movies[bad_word][1]:
            number_of_bad_words = number_of_bad_words + \
            inappropriate_words_in_0_label_movies[bad_word][1][movie_name]
    percentage_of_bad_words_in_0_labeld_movies[movie_name] = number_of_bad_words/movie_words_count

In [99]:
#with open('C:/Users/Abdo/Project/pickle_files/percentage_of_bad_words_in_0_labeld_movies.pkl', 'wb') as output:
    #pickle.dump(percentage_of_bad_words_in_0_labeld_movies, output)

In [26]:
pickle_in = open('C:/Users/Abdo/Project/pickle_files/percentage_of_bad_words_in_0_labeld_movies.pkl','rb')
percentage_of_bad_words_in_0_labeld_movies = pickle.load(pickle_in)

In [27]:
# turn the dictionary into list to sort it
# [ (percentage_of_bad_words_in_movie, movie_name) ]
percentage_of_bad_words_in_0_labeld_movies_list = []
for movie_name in percentage_of_bad_words_in_0_labeld_movies:
    percentage_of_bad_words_in_0_labeld_movies_list.append\
        ( (percentage_of_bad_words_in_0_labeld_movies[movie_name], movie_name) )

In [28]:
percentage_of_bad_words_in_0_labeld_movies_list.sort()

In [29]:
print(percentage_of_bad_words_in_0_labeld_movies_list[-1])

(0.019112874143526866, 'PG_4918')
